# Example: Produce *run* for Stocks

In [2]:
%load_ext sql
%sql postgresql://ftian@localhost/ftian

'Connected: ftian@ftian'

First we generate some data.   The following CTAS will populate a
table, with 4 Stocks (S1, S2, S3, S4) with 200 days of price.  
Price are generated, assume all of them start at $100.  Each day
a stock may randomly go up (80 percent chance) or down (20 percent) 

In [31]:
gensql = """
DROP TABLE IF EXISTS stock;
CREATE TABLE stock AS select symbol, day, price FROM (
select dg_utils.transducer_column_text(1) as symbol,
         dg_utils.transducer_column_int4(2) as day,
         dg_utils.transducer_column_float8(3) as price, 
         dg_utils.transducer($PHI$PhiExec go
         
// BEGIN INPUT TYPES
// i int32 
// END INPUT TYPES
//
// BEGIN OUTPUT TYPES
// symbol string
// day int32
// price float64
// END OUTPUT TYPES
//

package main

import (
        "fmt"
        "math/rand"
)

func main() {
        for rec := NextInput(); rec != nil; rec = NextInput() {
                i, _ := rec.Get_i()
                symbol := fmt.Sprintf("S%d", i)
                p := 100.0
                for n:=0; n<200; n++ {
                        var outrec OutRecord
                        outrec.Set_symbol(symbol)
                        outrec.Set_day(int32(n))
                        delta := rand.Float64() - 0.2
                        p += delta
                        outrec.Set_price(p)
                        WriteOutput(&outrec)
                }
        }
        WriteOutput(nil)
}
$PHI$), t.*
from ( select i::int from generate_series(1, 4) i ) t
) foo
"""

In [38]:
%sql $gensql 

Done.
800 rows affected.


[]

In [35]:
rows = %sql select * from stock where day > 195
print(rows)

16 rows affected.
+--------+-----+------------------+
| symbol | day |      price       |
+--------+-----+------------------+
|   S2   | 196 | 163.499978446567 |
|   S2   | 197 | 163.638955834962 |
|   S2   | 198 | 163.911447887214 |
|   S2   | 199 | 164.697442247222 |
|   S4   | 196 | 157.693863767325 |
|   S4   | 197 | 158.059793053449 |
|   S4   | 198 | 157.93169432978  |
|   S4   | 199 | 158.021571454425 |
|   S1   | 196 | 154.113343484876 |
|   S1   | 197 | 154.912502975096 |
|   S1   | 198 | 155.124043338317 |
|   S1   | 199 | 155.035717975082 |
|   S3   | 196 | 149.880174052084 |
|   S3   | 197 | 150.477918984064 |
|   S3   | 198 | 150.823305593446 |
|   S3   | 199 | 151.028042443998 |
+--------+-----+------------------+


Process stock data to generate runs.

In [3]:
runsql = """
WITH stockrun AS (
select dg_utils.transducer_column_text(1) as symbol,
       dg_utils.transducer_column_int4(2) as d0,
       dg_utils.transducer_column_float8(3) as p0,
       dg_utils.transducer_column_int4(4) as d1,
       dg_utils.transducer_column_float8(5) as p1,
       dg_utils.transducer($PHI$PhiExec go
// 
// BEGIN INPUT TYPES
// symbol string
// day int32
// price float64
// END INPUT TYPES
//
// BEGIN OUTPUT TYPES
// symbol string
// start int32
// startprice float64
// end int32
// endprice float64
// END OUTPUT TYPES
//

package main

func main() {
        var outrec *OutRecord
        for rec := NextInput(); rec != nil; rec = NextInput() {
                symbol, _ := rec.Get_symbol()
                day, _ := rec.Get_day()
                price, _ := rec.Get_price()

                if day == 0 {
                        if outrec != nil {
                                WriteOutput(outrec)
                        }
                        outrec = new(OutRecord)
                        outrec.Set_symbol(symbol)
                        outrec.Set_start(day)
                        outrec.Set_startprice(price)
                        outrec.Set_end(day)
                        outrec.Set_endprice(price)
                } else {
                        // Check if it is a run, either up or down.
                        isuprun := price >= outrec.GetValue_endprice() && outrec.GetValue_endprice() >= outrec.GetValue_startprice()
                        isdownrun := price <= outrec.GetValue_endprice() && outrec.GetValue_endprice() <= outrec.GetValue_startprice()
                        if isuprun || isdownrun {
                                outrec.Set_end(day)
                                outrec.Set_endprice(price)
                        } else {
                                oldrec := outrec
                                outrec = new(OutRecord)
                                outrec.Set_symbol(symbol)
                                outrec.Set_start(oldrec.GetValue_end())
                                outrec.Set_startprice(oldrec.GetValue_endprice())
                                outrec.Set_end(day)
                                outrec.Set_endprice(price)
                                WriteOutput(oldrec) 
                        }
                }
        }

        if outrec != nil {
                WriteOutput(outrec)
        }
        WriteOutput(nil)
}
$PHI$), t.isym, t.iday, t.iprice
from (
    select row_number() over (partition by symbol order by day), 
    symbol as isym, day as iday, price as iprice from stock
) t 
)

select symbol, max(d1 - d0) from stockrun where p1 < p0 group by symbol
"""

In [4]:
rows = %sql $runsql
print(rows)

4 rows affected.
+--------+-----+
| symbol | max |
+--------+-----+
|   S4   |  3  |
|   S2   |  2  |
|   S1   |  3  |
|   S3   |  4  |
+--------+-----+
